In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [11]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [12]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

# opt = torch_numopt.ConjugateGradient(model=model, lr=1e-4, line_search_method="const", cg_method="PR")
opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein", cg_method="PR")


all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.5963084697723389
epoch:  1, loss: 0.27325356006622314
epoch:  2, loss: 0.14448611438274384
epoch:  3, loss: 0.08463037014007568
epoch:  4, loss: 0.0563373826444149
epoch:  5, loss: 0.043102383613586426
epoch:  6, loss: 0.03699345514178276
epoch:  7, loss: 0.034211233258247375
epoch:  8, loss: 0.0329512394964695
epoch:  9, loss: 0.03237833082675934
epoch:  10, loss: 0.03211156278848648
epoch:  11, loss: 0.031985148787498474
epoch:  12, loss: 0.031922705471515656
epoch:  13, loss: 0.031887587159872055
epoch:  14, loss: 0.03183499723672867
epoch:  15, loss: 0.03176741302013397
epoch:  16, loss: 0.03172875940799713
epoch:  17, loss: 0.031658802181482315
epoch:  18, loss: 0.031584758311510086
epoch:  19, loss: 0.031568530946969986
epoch:  20, loss: 0.031408678740262985
epoch:  21, loss: 0.031320881098508835
epoch:  22, loss: 0.031252212822437286
epoch:  23, loss: 0.03106769174337387
epoch:  24, loss: 0.03096788562834263
epoch:  25, loss: 0.03088846057653427
epoch:  26, lo

In [13]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.7631731348323153
Test metrics:  R2 = 0.8241349821170552


In [14]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

# opt = torch_numopt.ConjugateGradient(model=model, lr=1e-4, line_search_method="const", cg_method="FR")
opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein", cg_method="FR")

all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.1518639326095581
epoch:  1, loss: 0.09925659000873566
epoch:  2, loss: 0.06943614035844803
epoch:  3, loss: 0.05270099639892578
epoch:  4, loss: 0.043459419161081314
epoch:  5, loss: 0.03843265771865845
epoch:  6, loss: 0.035732947289943695
epoch:  7, loss: 0.034297212958335876
epoch:  8, loss: 0.03353939577937126
epoch:  9, loss: 0.03314173221588135
epoch:  10, loss: 0.03293376788496971
epoch:  11, loss: 0.032825078815221786
epoch:  12, loss: 0.03276817873120308
epoch:  13, loss: 0.03273823484778404
epoch:  14, loss: 0.03272224962711334
epoch:  15, loss: 0.032713521271944046
epoch:  16, loss: 0.032708510756492615
epoch:  17, loss: 0.03270817920565605
epoch:  18, loss: 0.032700322568416595
epoch:  19, loss: 0.03269578143954277
epoch:  20, loss: 0.0326940156519413
epoch:  21, loss: 0.032687004655599594
epoch:  22, loss: 0.0326828770339489
epoch:  23, loss: 0.03267928957939148
epoch:  24, loss: 0.03267284482717514
epoch:  25, loss: 0.032668955624103546
epoch:  26, loss

In [15]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.7791152601841428
Test metrics:  R2 = 0.8343205966773746
